In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os
import random
from tqdm.notebook import tqdm
from pathlib import Path
from collections import Counter
import requests
import json
from googlefinance import getQuotes
from bs4 import BeautifulSoup
import requests, sys, lxml.html
# from finquant.portfolio import build_portfolio

# import yfinance as yf
from yahooquery import Ticker

from src import URLS, extract_html, extract_table, load_etf_snapshot, \
    load_etf_analyst, load_etf_pie, load_etf_holdings, get_yf, get_ticker, \
    get_recommendations, get_fund_holdings, get_fund_sectors, get_type, get_fund_categories, \
    load_stock_snapshot, load_stock_contained
from src.fidelity import _stock_company

In [2]:
names = set(['MSFT', 'SNE', 'AMD', 'TSLA', 'NIO', 'LI', 
         'ARKK', 'VONG', 'IBB', 'SPY', 'ESPO', 'PHO', 'ACES', 'SCHO', 'LQD', 'ICVT', 
        'ACES', 'AGG', 'ASHR', 'BIDU'])
names.update(['ACES', 'AGG', 'AMD', 'AMZN', 'ASHR', 'BABA', 'BIDU', 'CIBR', 'CNYA', 'DIS', 'EMXC', 'ESPO', 
              'EWC', 'EWG', 'EWY', 'FB', 'FHLC', 'FIDU', 'FIPDX', 'FLJP', 'FNCL',
               'GOOG', 'IAGG', 'IAU', 'IBB', 'ICLN', 'IGOV', 'KWEB', 'LQD', 'MCHI', 
              'MINT', 'MSFT', 'NFLX', 'NIO', 'NTDOY', 'NVDA', 'PHO', 'PYPL', 'SCHO', 'SKYY', 'SNE', 'SOCL', 
              'TCEHY', 'TSLA', 'VCLT', 'VCSH', 'VGK', 'VGLT', 'VONG', 'XITK'])
names.update(['FSGGX', 'FSMAX', 'FSPSX', 'FUAMX', 'FZIPX', 'FZROX',])

names = list(names)

cats = {}
for n in names:
    t = get_type(n)
    if t not in cats:
        cats[t] = []
    cats[t].append(n)

name = random.choice(names)
name

'MINT'

In [3]:
cats

{'ETF': ['FIDU',
  'ASHR',
  'EWC',
  'KWEB',
  'SPY',
  'VCLT',
  'IBB',
  'VGLT',
  'VONG',
  'FHLC',
  'CNYA',
  'LQD',
  'SKYY',
  'EMXC',
  'XITK',
  'ICLN',
  'IAGG',
  'MCHI',
  'FNCL',
  'SOCL',
  'VCSH',
  'EWG',
  'CIBR',
  'IGOV',
  'SCHO',
  'VGK',
  'ESPO',
  'PHO',
  'MINT',
  'IAU',
  'AGG',
  'ACES',
  'EWY',
  'ARKK',
  'ICVT',
  'FLJP'],
 'EQUITY': ['AMZN',
  'NVDA',
  'SNE',
  'NFLX',
  'BIDU',
  'DIS',
  'NTDOY',
  'GOOG',
  'AMD',
  'LI',
  'TSLA',
  'PYPL',
  'MSFT',
  'NIO',
  'TCEHY',
  'FB',
  'BABA'],
 'MUTUALFUND': ['FSMAX', 'FZROX', 'FIPDX', 'FSGGX', 'FSPSX', 'FZIPX', 'FUAMX']}

In [102]:
# print(URLS['stocks']['snapshot'].format(ticker='GOOG'))
print(URLS['etfs']['composition'].format(ticker='SCHO'))
# for name in cats['EQUITY']:
# #     print(URLS['etf']['holdings'].format(ticker=etf))
#     print(URLS['stocks']['snapshot'].format(ticker=name))

https://eresearch.fidelity.com/eresearch/evaluate/snapshot.jhtml?symbols=GOOG
https://screener.fidelity.com/ftgw/etf/goto/snapshot/portfolioComposition.jhtml?symbols=SCHO


In [5]:
tk = get_ticker(name)
tk.symbols[0]

'EWG'

In [6]:
recs = get_recommendations(name)
recs

['EWU', 'EWQ', 'EWP', 'EWI', 'EWL']

In [35]:
get_type(name)

'ETF'

In [37]:
# [get_type(n) for n in names]

In [49]:
assets = get_fund_categories(name)
assets

{'stocks': 0.9958, 'bonds': 0.0}

In [3]:
# root = Path('raw/27-12-20/')
root = Path('raw/')
root = Path('raw/stocks')
root = Path('raw/dates/6-1-21/equity')
root

WindowsPath('raw/dates/6-1-21/equity')

In [7]:
# paths = list(root.glob('etf*'))
paths = sorted(list(root.glob('snapshot*')))
# paths = sorted(list(root.glob('*Details*')))
# paths = sorted(list(root.glob('etfholdings*')))
path = paths[18]
len(paths), path

(20, WindowsPath('raw/dates/6-1-21/equity/snapshot (9).html'))

In [8]:
bs = extract_html(path)

In [9]:
ticker = bs.findAll('title')[0].contents[0].split(' | ')[0].strip()
ticker

'DIS'

In [10]:
# bs

In [11]:
tbs = bs.findAll("table")
len(tbs)

33

In [20]:
table = tbs[13]

In [29]:

sector, industry = [x.text.strip() for x in table.find_all('span', {'class': 'right'})[:2]]

elms = table.find_all('p')
info = [el.text for el in elms]
start_idx = [i for i, x in enumerate(info) if x.startswith('Executives')][0] + 1
end_idx = [i for i, x in enumerate(info) if x.startswith('Environmental, Social, and')][0]
*people, addr, phone = elms[start_idx:end_idx]

In [30]:
sector, industry

('Communication Services', 'Entertainment')

In [31]:
start_idx, end_idx

(12, 14)

In [35]:
elms[8:]

[<p>The Walt Disney Company, together with its subsidiaries, operates as an entertainment company worldwide. The company's Media Networks segment operates domestic cable networks under the Disney, ESPN, Freeform, FX, and National Geographic brands; and television broadcast network under the ABC brand, as well as eight domestic television stations. This segment is also involved in the television production and distribution. Its Parks, Experiences and Products segment operates theme parks and resorts, such as Walt Disney World Resort in Florida; Disneyland Resort in California; Disneyland Paris; Hong Kong Disneyland Resort; and Shanghai Disney Resort; Disney Cruise Line, Disney Vacation Club, National Geographic Expeditions, and Adventures by Disney and Aulani, a Disney resort and spa in Hawaii, as well as licenses its intellectual property to a third party for the operations of the Tokyo Disney Resort in Japan. The company's Studio Entertainment segment produces and distributes motion p

In [18]:
company = _stock_company(tbs[13])

[<p>The Walt Disney Company (<span fmr-attach-component="fmr.qnr.components.snapshot.Mini" fmr-param-symbol="DIS">DIS</span>) will discuss fiscal first quarter 2021 financial results via a live audio webcast beginning at 4:30 p.m. ET / 1:30 p.m. PT on Thursday, February 11, 2021. Results will be released after the close of regular trading on February 11, 2021. To listen to the webcast, please visit www.disney.com/investors. The webcast presentation will be archived.</p>, <p>In a virtual gathering of chief executives convened to discuss political turbulence stemming from the 2020 election, leaders of some of the largest U.S. companies said they were considering withholding donations to Republican lawmakers seeking to impede the presidential transition. During an hourlong meeting Tuesday, a group of about three dozen chief executives discussed divisions in the country, efforts to thwart the presidential transition and what voice they...</p>, <p>http://archive.fast-edgar.com/20210105/ARBZ

TypeError: 'NoneType' object is not callable

In [113]:
# tb = tbs[25]
table = tbs[1]

In [171]:
extract_table(tbs[2])

[['Top 10', '21.46%'], ['Other', '78.54%']]

In [112]:
extract_table(table)

[['Asset Classification', 'Fixed Income'],
 ['Investment Philosophy', 'Passively Managed']]

In [99]:
# table

In [100]:
from src.fidelity import _fmt_human, _fmt_percent, _fmt_online

In [146]:
table

<table border="0" cellpadding="0" cellspacing="0">
<tr>
<th>Asset Classification</th>
<td>Fixed Income</td>
</tr>
<tr>
<th>Investment Philosophy</th>
<td>Passively Managed</td>
</tr>
</table>

In [46]:
# data = extract_table(table)

In [45]:
# bs.find_all('img', {'alt':"", 'height':"15", 'src':"https://www.fidelity.com/common/always-cached/etf/10-07/images/magnify2.gif", 
#                     'width':"17"})
els = bs.find('div', {'class':"spacing-div_15X0"}) # div class="spacing-div_15X0"
# els = bs.find_all('h3')
els

<div class="spacing-div_15X0"></div>

In [145]:
els[2]

<div class="header-with-borders">
<h3><a class="simple-popup" href="http://research2.fidelity.com/fidelity/screeners/etf/etfholdings.asp?symbol=SCHO&amp;view=Region" title="View Regions by Weight"><img alt="" height="15" src="https://www.fidelity.com/common/always-cached/etf/10-07/images/magnify2.gif" width="17"/> 
						Regional Exposure</a></h3>
</div>

In [168]:
missing = [x.text.strip().replace('Sorry : ', '').replace(' is not available for this symbol.', '') 
           for x in bs.find_all('div', {'class':"error-message"})]
missing

['Regional Exposure']

In [134]:
els[0].text.strip().split('\r')[0]

'Holdings'

In [139]:
tbs[4]

<table border="0" cellpadding="0" cellspacing="0" class="composition-table">
<tr>
<td class="pie-sector-indicator">
<div style="width:7px; height:7px; background-color:#F7A600;"></div>
</td>
<th class="pie-sector-name" scope="row">Government</th>
<td class="pie-sector-area">99.86%</td>
</tr>
<tr class="alt-row-color">
<td class="pie-sector-indicator">
<div style="width:7px; height:7px; background-color:#003063;"></div>
</td>
<th class="pie-sector-name" scope="row">Corporate</th>
<td class="pie-sector-area">0.00%</td>
</tr>
<tr>
<td class="pie-sector-indicator">
<div style="width:7px; height:7px; background-color:#E76108;"></div>
</td>
<th class="pie-sector-name" scope="row">Mortgage</th>
<td class="pie-sector-area">0.00%</td>
</tr>
<tr class="alt-row-color">
<td class="pie-sector-indicator">
<div style="width:7px; height:7px; background-color:#529E00;"></div>
</td>
<th class="pie-sector-name" scope="row">Municipal</th>
<td class="pie-sector-area">0.00%</td>
</tr>
<tr>
<td class="pie-se

In [141]:
tbs[5]

<table border="0" cellpadding="0" cellspacing="0" class="composition-table">
<tr>
<td class="pie-sector-indicator">
<div style="width:7px; height:7px; background-color:#003063;"></div>
</td>
<th class="pie-sector-name" scope="row">United States</th>
<td class="pie-sector-area">100.00%</td>
</tr>
</table>

In [140]:
extract_table(tbs[4])

[['Government', '99.86%'],
 ['Corporate', '0.00%'],
 ['Mortgage', '0.00%'],
 ['Municipal', '0.00%'],
 ['Preferred', '0.00%']]

In [68]:
# for path in paths:
#     bs = extract_html(path)
#     tbs = bs.findAll("table")
#     tb = tbs[25]
#     rows = extract_table(tb)
#     print(rows[0])

In [40]:
txt = str(table)
offset = 0 #+ 91817
idx = txt[offset:].find('Robert A. Chapek')+offset
idx

13228

In [41]:
rn = 500
txt[idx-min(rn, idx):idx+min(rn,len(txt)-idx)]

'/eresearch/gotoBL/evaluate/snapshot.jhtml?symbols=DIS&amp;opt=1">Visit the new Stock Dashboard</a>\n</div>\n<div class="spacing-div_12X0"></div>\n<div class="dark-grey-hr"><div class="hr-for-ie"></div></div>\n<div class="spacing-div_15X0"></div>\n<div class="spacing-div_20X0"></div>\n<p><span class="bold">Full Time Employees:\xa0203,000</span></p>\n<div class="spacing-div_10X0"></div>\n<p><span class="bold">Executives</span></p>\n<div class="spacing-div_10X0"></div>\n<div>\n<ul>\n<div id="executives">\n<li>Mr. Robert A. Chapek,\xa0Chief Executive</li><li>Mr. Robert A. Chapek,\xa0Director</li><li>Mr. Robert A. Iger,\xa0Chairman</li><li>Mr. Robert A. Iger,\xa0Director</li><li>Ms. Christine Mary McCarthy,\xa0Chief Finance Officer</li>\n<div class="spacing-div_10X0"></div>\n<a href="javascript:viewMore(\'executives\');">\n<img height="12" src="https://www.fidelity.com/common/application/etf/14.10/images/plus_blue.gif" width="12"/>\r\n\t\t\t\t\t\tView more\r\n\t\t\t\t\t</a>\n</div>\n<div c

In [121]:
# len(str(tb))

In [120]:
# str(tb)[34000:35500]

In [173]:
root = Path('raw/stocks/')
# paths = root.glob('analyst*')
# paths = list(root.glob('snapshot*'))
paths = list(root.glob('snap*'))
len(paths)

17

In [152]:
# out = dict( load_etf_holdings(path) for path in tqdm(list(paths)))
out = dict( load_stock_snapshot(path) for path in tqdm(list(paths)))
len(out)

  0%|          | 0/17 [00:00<?, ?it/s]

17

In [31]:
# out['GOOG']

In [32]:
corps = {ticker:info['company'] for ticker, info in out.items()}
# corps

In [30]:
[c['name'] for c in corps.values()]

['Alphabet Inc',
 'Facebook Inc',
 'Advanced Micro Devices Inc',
 'Netflix Inc',
 'Nvidia Corp',
 'Amazoncom Inc',
 'Nintendo Co Ltd',
 'Li Auto Inc',
 'Microsoft Corp',
 'Walt Disney Co',
 'Alibaba Group Holding Ltd',
 'Tencent Holdings Ltd',
 'Nio Inc',
 'Tesla Inc',
 'Baidu Inc',
 'Paypal Holdings Inc',
 'Sony Corp']

In [188]:
root = Path('raw/27-12-20/')
# paths = root.glob('analyst*')
# paths = list(root.glob('snapshot*'))
# paths = list(root.glob('snap*'))
paths = list(root.glob('port*'))
path = paths[5]
len(paths), path

(36, WindowsPath('raw/27-12-20/portfolioComposition (14).html'))

In [189]:
# out = dict( load_etf_snapshot(path) for path in tqdm(list(paths)))
out = dict( load_etf_pie(path) for path in tqdm(list(paths)))
len(out)

  0%|          | 0/36 [00:00<?, ?it/s]

36

In [190]:
out.keys()

dict_keys(['FIDU', 'IGOV', 'SOCL', 'FLJP', 'EWG', 'SCHO', 'VONG', 'VCLT', 'EMXC', 'CNYA', 'AGG', 'KWEB', 'MCHI', 'VGLT', 'CIBR', 'ASHR', 'ICVT', 'ICLN', 'SKYY', 'FNCL', 'EWC', 'IBB', 'SPY', 'ACES', 'IAU', 'ARKK', 'IAGG', 'MINT', 'ESPO', 'XITK', 'FHLC', 'EWY', 'VCSH', 'LQD', 'PHO', 'VGK'])

In [194]:
# ticker, data = load_etf_pie(path)
# ticker, data.keys()

In [196]:
out['SPY']

{'Asset Classification': 'Equity',
 'Investment Philosophy': 'Passively Managed',
 'Holdings': {'Apple Inc': 6.37,
  'Microsoft Corp': 5.39,
  'Amazon.com Inc': 4.49,
  'Facebook Inc': 2.21,
  'Alphabet Inc': 1.72,
  'Berkshire Hathaway Inc': 1.51,
  'Johnson & Johnson': 1.27,
  'JPMorgan Chase & Co': 1.2,
  'Visa Inc': 1.18},
 'Sector Exposure': {'Information Technology': 27.54,
  'Health Care': 13.61,
  'Consumer Discretionary': 11.35,
  'Communication Services': 11.0,
  'Financials': 10.39,
  'Industrials': 8.7,
  'Consumer Staples': 6.74,
  'Utilities': 2.88,
  'Materials': 2.69,
  'Real Estate': 2.54,
  'Energy': 2.29},
 'Industry Exposure': {'Software': 8.71,
  'Technology Hardware, Storage & Peripherals': 6.65,
  'Interactive Media & Services': 5.8,
  'IT Services': 5.54,
  'Semiconductors & Semiconductor Equipment': 5.23,
  'Internet & Direct Marketing Retail': 5.0,
  'Pharmaceuticals': 3.9,
  'Health Care Equipment & Supplies': 3.75,
  'Banks': 3.75,
  'Capital Markets': 2.73,

In [1]:
raw = '''ASML	ASML Holding NV	5.48
MC	LVMH Moet Hennessy Louis Vuitton SE	3.92
TTE	TotalEnergies SE	2.51
SAP	SAP SE	2.29
SAN	Sanofi SA	2.29
SIE	Siemens AG	2.09
OR	L'Oreal SA	1.98
ALV	Allianz SE	1.92
SU	Schneider Electric SE	1.79
AI	Air Liquide SA	1.64
AIR	Airbus SE	1.41
BAYN	Bayer AG	1.33
IBE	Iberdrola SA	1.27
BNP	BNP Paribas Act. Cat.A	1.27
DTE	Deutsche Telekom AG	1.19
MBG	Mercedes-Benz Group AG	1.19
SAN	Banco Santander SA	1.17
CS	AXA SA	1.12
DG	Vinci SA	1.09
ENEL	Enel SpA	1.07
EL	Essilorluxottica	1.04
BAS	Basf SE	1.03
ABI	Anheuser-Busch InBev SA/NV	1.02
PRX	Prosus NV Ordinary Shares - Class N	0.99
KER	Kering SA	0.94
DPW	Deutsche Post AG	0.94
RI	Pernod Ricard SA	0.91
RMS	Hermes International SA	0.89
IFX	Infineon Technologies AG	0.88
ADS	adidas AG	0.88
INGA	ING Groep NV	0.81
SAF	Safran SA	0.80
ADYEN	Adyen NV	0.78
BBVA	Banco Bilbao Vizcaya Argentaria SA	0.75
ISP	Intesa Sanpaolo	0.75
MUV2	Munchener Ruckversicherungs-Gesellschaft AG	0.74
ENI	Eni SpA	0.73
BN	Danone SA	0.71
CAP	Capgemini SE	0.71
VNA	Vonovia SE	0.68
DB1	Deutsche Boerse AG	0.67
AD	Koninklijke Ahold Delhaize NV	0.66
STLA	Stellantis NV	0.65
DSY	Dassault Systemes SE	0.65
VOW3	Volkswagen AG Participating Preferred	0.63
DSM	Koninklijke DSM NV	0.62
CRH	CRH PLC	0.61
SGO	Compagnie de Saint-Gobain SA	0.60
STM	STMicroelectronics NV	0.59
NOKIA	Nokia Oyj	0.58
AMS	Amadeus IT Group SA A	0.58
BMW	Bayerische Motoren Werke AG	0.57
PHIA	Koninklijke Philips NV	0.55
RWE	RWE AG	0.55
WKL	Wolters Kluwer NV	0.55
RACE	Ferrari NV	0.54
MRK	Merck KGaA	0.53
DBK	Deutsche Bank AG	0.52
TEF	Telefonica SA	0.51
EOAN	E.ON SE	0.51
G	Assicurazioni Generali	0.50
LR	Legrand SA	0.50
HEIA	Heineken NV	0.49
CLNX	Cellnex Telecom SA	0.49
SAMPO	Sampo Oyj Class A	0.48
ENGI	Engie SA	0.47
ITX	Industria De Diseno Textil SA	0.47
ORA	Orange SA	0.46
ML	Cie Generale des Etablissements Michelin SA	0.45
UCG	UniCredit SpA	0.45
TEP	Teleperformance SE	0.44
GLE	Societe Generale SA	0.43
VIE	Veolia Environnement SA	0.41
MT	ArcelorMittal SA	0.41
UMG	Universal Music Group NV	0.38
REP	Repsol SA	0.38
FLTR	Flutter Entertainment PLC	0.38
NESTE	Neste Corp	0.38
Eur Cash	0.36
KBC	KBC Group SA/NV	0.36
Usd Cash	0.35
SHL	Siemens Healthineers AG Registered Shares	0.35
KRZ	Kerry Group PLC Class A	0.35
KNEBV	KONE Oyj Class B	0.35
XTSLA	BlackRock Cash Funds Treasury SL Agency	0.35
ASM	ASM International NV	0.34
UPM	UPM-Kymmene Oyj	0.34
AKZA	Akzo Nobel NV	0.32
CNHI	CNH Industrial NV	0.32
SY1	Symrise AG	0.31
KRX	Kingspan Group PLC	0.30
CABK	CaixaBank SA	0.30
UCB	UCB SA	0.30
FRE	Fresenius SE & Co KGaA	0.30
PAH3	Porsche Automobil Holding SE Participating Preferred	0.29
ACA	Credit Agricole SA	0.29
ARGX	argenx SE	0.28
NN	NN Group NV	0.27
EDP	EDP - Energias de Portugal SA	0.27
FME	Fresenius Medical Care AG & Co. KGaA	0.27
PUB	Publicis Groupe SA	0.27
HO	Thales	0.26
ERF	Eurofins Scientific SE	0.26
CA	Carrefour	0.26
FER	Ferrovial SA	0.26
AENA	Aena SME SA	0.25
EBS	Erste Group Bank AG.	0.25
BNR	Brenntag SE	0.25
MTX	MTU Aero Engines AG	0.24
EDEN	Edenred SA	0.24
TRN	Terna SpA	0.24
HEN3	Henkel AG & Co KGaA Participating Preferred	0.23
SRG	Snam SpA	0.23
KPN	Koninklijke KPN NV	0.23
SRT3	Sartorius AG Participating Preferred	0.23
MONC	Moncler SpA	0.23
DTG	Daimler Truck Holding AG Ordinary Shares	0.23
STERV	Stora Enso Oyj Class R	0.22
DIM	Sartorius Stedim Biotech SA	0.22
ZAL	Zalando SE	0.22
GBLB	Groupe Bruxelles Lambert SA	0.22
QGEN	Qiagen NV	0.22
SK3	Smurfit Kappa Group PLC	0.22
BEI	Beiersdorf AG	0.21
WLN	Worldline SA	0.21
HNR1	Hannover Rueck SE	0.20
ATL	Atlantia SpA	0.20
VIV	Vivendi SE	0.20
CCEP	Coca-Cola Europacific Partners PLC	0.20
1COV	Covestro AG	0.19
IMCD	IMCD NV	0.19
AGN	Aegon NV	0.19
URW	Unibail-Rodamco-Westfield Act. SIIC ET STES FONC.EUROP.	0.19
FBK	FinecoBank SpA	0.18
ENR	Siemens Energy AG Ordinary Shares	0.18
HEIO	Heineken Holding NV	0.18
PUM	Puma SE	0.18
AGS	Ageas SA/ NV	0.17
PRY	Prysmian SpA	0.17
FGR	Eiffage SA	0.17
ELISA	Elisa Oyj Class A	0.17
HEI	HeidelbergCement AG	0.17
UMI	Umicore SA	0.17
BVI	Bureau Veritas SA	0.17
LEG	LEG Immobilien SE	0.16
EXO	EXOR NV	0.16
VOW	Volkswagen AG	0.16
FORTUM	Fortum Oyj	0.16
EN	Bouygues	0.16
CON	Continental AG	0.16
GET	Getlink SE	0.16
ENX	Euronext NV	0.15
CBK	Commerzbank AG	0.15
KESKOB	Kesko Oyj Class B	0.15
HFG	HelloFresh SE Bearer Shares	0.15
ALO	Alstom SA	0.15
EDPR	EDP Renovaveis SA	0.15
AKE	Arkema SA	0.15
SOLB	Solvay SA	0.14
SW	Sodexo	0.14
VER	Verbund AG Class A	0.14
RAND	Randstad NV	0.14
DHER	Delivery Hero SE	0.14
TEN	Tenaris SA	0.14
OMV	OMV AG	0.14
ACS	ACS Actividades de Construccion y Servicios SA	0.14
ELE	Endesa SA	0.14
HEN	Henkel AG & Co KGaA	0.14
AER	AerCap Holdings NV	0.13
JMT	Jeronimo Martins SGPS SA	0.13
REE	Red Electrica Corporacion SA	0.13
AFX	Carl Zeiss Meditec AG	0.13
MB	Mediobanca SpA	0.13
G1A	GEA Group AG	0.13
EVK	Evonik Industries AG	0.12
PST	Poste Italiane SpA	0.12
NTGY	Naturgy Energy Group SA	0.12
NEXI	Nexi SpA	0.12
CPR	Davide Campari-Milano NV Az nom Post Frazionamento	0.12
GALP	Galp Energia SGPS SA	0.12
TKWY	Just Eat Takeaway.com NV	0.12
AC	Accor SA	0.11
ABN	ABN AMRO Bank NV	0.11
AMP	Amplifon SpA Az nom Post Frazionamento	0.11
KBX	Knorr-Bremse AG Bearer Shares	0.11
NEM	Nemetschek SE	0.11
SOF	Sofina SA	0.11
AT1	Aroundtown SA	0.11
GRF	Grifols SA	0.11
GFC	Gecina Nom	0.11
LI	Klepierre	0.11
ORNBV	Orion Oyj Class B	0.10
LHA	Deutsche Lufthansa AG	0.10
RNO	Renault SA	0.10
REC	Recordati SpA Az nom Post raggruppamento	0.10
ADP	Aeroports de Paris SA	0.09
BIM	BioMerieux SA	0.09
BMW3	Bayerische Motoren Werke AG Participating Preferred	0.09
EDF	Electricite de France SA	0.09
BC8	Bechtle AG	0.09
ELI	Elia Group	0.09
BOL	Bollore SE	0.09
RCO	Remy Cointreau	0.09
G24	Scout24 SE	0.09
IPN	Ipsen SA	0.09
KGX	KION GROUP AG	0.09
DIA	Diasorin SpA	0.08
AM	Dassault Aviation SA	0.08
SK	SEB SA	0.08
ENG	Enagas SA	0.08
FDJ	La Francaise Des Jeux SA Ordinary Shares	0.08
AMUN	Amundi SA	0.08
CNP	CNP Assurances	0.08
UBI	Ubisoft Entertainment	0.08
SGRE	Siemens Gamesa Renewable Energy SA	0.08
FR	Valeo SA	0.08
WRT1V	Wartsila Corp	0.08
COV	Covivio SA	0.08
UTDI	United Internet AG	0.07
RF	Eurazeo SE	0.07
VOE	voestalpine AG	0.07
LXS	Lanxess AG	0.07
RAA	Rational AG	0.07
INW	Infrastrutture Wireless Italiane SpA	0.07
TIT	Telecom Italia SpA	0.07
MF	Wendel SE	0.06
PROX	Proximus SA	0.06
O2D	Telefonica Deutschland Holding AG	0.06
JDEP	JDE Peets NV Ordinary Shares	0.06
EO	Faurecia SE	0.06
ORP	Orpea SA	0.05
UN01	Uniper SE	0.05
FPE3	Fuchs Petrolub SE Participating Preferred	0.05
Cash Collateral Eur Jpfft	0.04
COLR	Etablissementen Franz Colruyt NV	0.04
RBI	Raiffeisen Bank International AG	0.04
INPST	Inpost SA Ordinary Shares	0.03'''.split('\n')
len(raw)

237

In [5]:
def line_parse(line):
    *ident, num = line.split('\t')
    if len(ident) == 2:
        tk, name = ident
    else:
        tk, name = ident[0], ident[0]
    return tk, name, float(num)

tks, names, amounts = zip(*map(line_parse,raw))
len(tks)

237

In [6]:
print(tks)

('ASML', 'MC', 'TTE', 'SAP', 'SAN', 'SIE', 'OR', 'ALV', 'SU', 'AI', 'AIR', 'BAYN', 'IBE', 'BNP', 'DTE', 'MBG', 'SAN', 'CS', 'DG', 'ENEL', 'EL', 'BAS', 'ABI', 'PRX', 'KER', 'DPW', 'RI', 'RMS', 'IFX', 'ADS', 'INGA', 'SAF', 'ADYEN', 'BBVA', 'ISP', 'MUV2', 'ENI', 'BN', 'CAP', 'VNA', 'DB1', 'AD', 'STLA', 'DSY', 'VOW3', 'DSM', 'CRH', 'SGO', 'STM', 'NOKIA', 'AMS', 'BMW', 'PHIA', 'RWE', 'WKL', 'RACE', 'MRK', 'DBK', 'TEF', 'EOAN', 'G', 'LR', 'HEIA', 'CLNX', 'SAMPO', 'ENGI', 'ITX', 'ORA', 'ML', 'UCG', 'TEP', 'GLE', 'VIE', 'MT', 'UMG', 'REP', 'FLTR', 'NESTE', 'Eur Cash', 'KBC', 'Usd Cash', 'SHL', 'KRZ', 'KNEBV', 'XTSLA', 'ASM', 'UPM', 'AKZA', 'CNHI', 'SY1', 'KRX', 'CABK', 'UCB', 'FRE', 'PAH3', 'ACA', 'ARGX', 'NN', 'EDP', 'FME', 'PUB', 'HO', 'ERF', 'CA', 'FER', 'AENA', 'EBS', 'BNR', 'MTX', 'EDEN', 'TRN', 'HEN3', 'SRG', 'KPN', 'SRT3', 'MONC', 'DTG', 'STERV', 'DIM', 'ZAL', 'GBLB', 'QGEN', 'SK3', 'BEI', 'WLN', 'HNR1', 'ATL', 'VIV', 'CCEP', '1COV', 'IMCD', 'AGN', 'URW', 'FBK', 'ENR', 'HEIO', 'PUM', 'A

In [7]:
pf = {tk:v for tk, v in zip(tks, amounts)}
len(pf), sum(pf.values())

(236, 98.41999999999996)

In [8]:
missing = ['MBG', 'CRH', 'NOKIA', 'SAMPO', 'MT', 'FLTR', 'NESTE', 'Eur Cash', 'Usd Cash', 'KRZ', 'KNEBV', 'XTSLA', 'UPM', 'KRX', 'STERV', 'QGEN', 'SK3', 'URW', 'ELISA', 'FORTUM', 'KESKOB', 'ELE', 'AER', 'ORNBV', 'WRT1V', 'Cash Collateral Eur Jpfft']


In [9]:
sum(pf[k] for k in missing)

8.249999999999998

In [11]:
print(amounts)

(5.48, 3.92, 2.51, 2.29, 2.29, 2.09, 1.98, 1.92, 1.79, 1.64, 1.41, 1.33, 1.27, 1.27, 1.19, 1.19, 1.17, 1.12, 1.09, 1.07, 1.04, 1.03, 1.02, 0.99, 0.94, 0.94, 0.91, 0.89, 0.88, 0.88, 0.81, 0.8, 0.78, 0.75, 0.75, 0.74, 0.73, 0.71, 0.71, 0.68, 0.67, 0.66, 0.65, 0.65, 0.63, 0.62, 0.61, 0.6, 0.59, 0.58, 0.58, 0.57, 0.55, 0.55, 0.55, 0.54, 0.53, 0.52, 0.51, 0.51, 0.5, 0.5, 0.49, 0.49, 0.48, 0.47, 0.47, 0.46, 0.45, 0.45, 0.44, 0.43, 0.41, 0.41, 0.38, 0.38, 0.38, 0.38, 0.36, 0.36, 0.35, 0.35, 0.35, 0.35, 0.35, 0.34, 0.34, 0.32, 0.32, 0.31, 0.3, 0.3, 0.3, 0.3, 0.29, 0.29, 0.28, 0.27, 0.27, 0.27, 0.27, 0.26, 0.26, 0.26, 0.26, 0.25, 0.25, 0.25, 0.24, 0.24, 0.24, 0.23, 0.23, 0.23, 0.23, 0.23, 0.23, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.21, 0.21, 0.2, 0.2, 0.2, 0.2, 0.19, 0.19, 0.19, 0.19, 0.18, 0.18, 0.18, 0.18, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.1